# Dependencies:

The code is in python3. 

You'll need to install the following libraries before being able to run. There may additional libraries you need to install to run.

Tensorflow (using 1.5 because newer versions broken on mac for some reason)

`pip3 install tensorflow==1.5`

Keras

`pip3 install keras`

OpenCV

`pip3 install opencv-python`

ImageAI

`pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl`

Pillow, Scipy, numpy, matplotlib
`pip3 install pillow scipy numpy matplotlib`

# Download TinyImageNet and Pre-trained YOLOv3 model weights

In [1]:
import requests, zipfile, io, os
import numpy as np

if not os.path.isdir(os.path.join(os.getcwd(), 'tiny-imagenet-200')):
    zip_file_url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
    print("Downloading Tiny ImageNet 200")
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    print("Extracting contents...")
    z.extractall()


def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

#Also download pretrained model file https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5
if not os.path.exists(os.path.join(os.getcwd(), 'yolo.h5')):
    print("Downloading pretrained YoloV3 model file...")
    download_file('https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5')


# Load bounding box information for each training image, organized by WordNet ID (wnid) 

This will be useful to parse information for specific images later

In [2]:
train_dir = os.path.join(os.getcwd(), 'tiny-imagenet-200', 'train')
#get all the subdirectories from train/, 
training_wnids = [x for x in os.listdir(train_dir)]

lines = [line.rstrip('\n').split('\t') for line in open('tiny-imagenet-200/words.txt')]
wnids_to_words = {line[0]:line[1] for line in lines }

train_bboxes = {} #{wnid: {image_name: bboxes}
for id in training_wnids:
    path = os.path.join(train_dir, id, id+"_boxes.txt")
    lines = [line.rstrip('\n').split('\t') for line in open(path)]
    train_bboxes[id] = { line[0]:np.array(line[1:], dtype=np.int) for line in lines }


# Load pretrained model using ImageAI and test on image

Model is trained on full imagenet, so using it here on tiny imagenet doesn't make a lot of sense, but it's only for testing purposes. 

Take in an image and output to "{testid}-test-out.jpg"

In [ ]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

test_id = 'n07920052'
test_img_num = 1
test_img_path = os.path.join(train_dir, test_id, 'images', test_id + '_' + str(test_img_num) + '.JPEG')

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "yolo.h5"))
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , test_img_path), output_image_path=os.path.join(execution_path , test_id + "-test-out.jpg"), minimum_percentage_probability=30)
pred_bbox = None
for eachObject in detections:
    pred_bbox = eachObject["box_points"]
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")
    
    
truth_bbox = train_bboxes[test_id][os.path.basename(test_img_path)]

# Calculate area overlap ratio of bounding box prediction vs. truth

In [95]:
# Give tuples/lists in form (xmin, ymin, xmax, ymax)

def overlap(a, b):  # returns None if rectangles don't intersect
    
    dx = min(a[2], b[2]) - max(a[0], b[0])
    dy = min(a[3], b[3]) - max(a[1], b[1])
    truth_area = (b[3]-b[1])*(b[2]-b[0])
    pred_area = (a[3]-a[1])*(a[2]-a[0])
#     print(truth_area)
    if (dx>=0) and (dy>=0):
        intersect = truth_area+pred_area-dx*dy
        return dx*dy / intersect
    

    
overlap(pred_bbox, truth_bbox)

# Scrap code

In [ ]:
# from keras.applications.resnet50 import ResNet50
# from keras.preprocessing import image
# from keras.applications.resnet50 import preprocess_input, decode_predictions
# import numpy as np

# model = ResNet50(weights='imagenet')

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# # decode the results into a list of tuples (class, description, probability)
# # (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# # Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]